In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Analisis inicial
Analizamos los tipos de datos que muestra el dataset.

In [3]:
df_train = pd.read_csv("salario.csv")

In [4]:
df_train.head()

,edad,trabajo,estudios,estado-civil,trabajo.1,posicion-familiar,etnia,sexo,ganancias_inversiones,perdidas_inversiones,horas-trabajo_semana,pais-origen,salario
0,39,State-gov,Bachelors,Never-married,Adm-clerical,No-en-familia,Blanco,Hombre,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Marido,Blanco,Hombre,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,No-en-familia,Blanco,Hombre,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Marido,Negro,Hombre,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Mujer,Negro,Mujer,0,0,40,Cuba,<=50K


In [5]:
df_train.info()
print('Testing Features shape: ', df_train.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27998 entries, 0 to 27997
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   edad                   27998 non-null  int64 
 1   trabajo                27998 non-null  object
 2   estudios               27998 non-null  object
 3   estado-civil           27998 non-null  object
 4   trabajo.1              27998 non-null  object
 5   posicion-familiar      27998 non-null  object
 6   etnia                  27998 non-null  object
 7   sexo                   27998 non-null  object
 8   ganancias_inversiones  27998 non-null  int64 
 9   perdidas_inversiones   27998 non-null  int64 
 10  horas-trabajo_semana   27998 non-null  int64 
 11  pais-origen            27998 non-null  object
 12  salario                27998 non-null  object
dtypes: int64(4), object(9)
memory usage: 2.8+ MB
Testing Features shape:  (27998, 13)


In [6]:
print(df_train.nunique())


edad                      72
trabajo                    9
estudios                  16
estado-civil               7
trabajo.1                 15
posicion-familiar          6
etnia                      5
sexo                       2
ganancias_inversiones    117
perdidas_inversiones      90
horas-trabajo_semana      94
pais-origen               42
salario                    2
dtype: int64


Haciendo un análisis principal del dataset, podemos obserbar que de las columnas 1 a la 7, junto a la 11 y 12 se tratan de variables categóricas.
En cuanto a las variables númericas luego deberemos tratar algun tipo de normalización o estandarización.

# Verificación de filas duplicadas
Vamos a comprobar si existen filas duplicadas en nuestro dataset, lo cual podría afectar al rendimiento y sesgar nuestro modelo.

In [18]:

duplicadas = df_train.duplicated().sum()
print(f"Número total de filas duplicadas: {duplicadas}")


porcentaje_duplicadas = (duplicadas / len(df_train)) * 100
print(f"Porcentaje de filas duplicadas: {porcentaje_duplicadas:.2f}%")


print("\nPrimeras filas duplicadas encontradas:")
df_train[df_train.duplicated()].head()

Número total de filas duplicadas: 2562
Porcentaje de filas duplicadas: 9.69%

Primeras filas duplicadas encontradas:


,edad,trabajo,estudios,estado-civil,trabajo.1,posicion-familiar,etnia,sexo,ganancias_inversiones,perdidas_inversiones,horas-trabajo_semana,pais-origen,salario
352,33,Private,Bachelors,Married-civ-spouse,Exec-managerial,Marido,Blanco,Hombre,0,0,40,United-States,>50K
391,27,Private,Bachelors,Never-married,Craft-repair,No-en-familia,Blanco,Hombre,0,0,50,United-States,<=50K
563,24,Private,HS-grad,Never-married,Handlers-cleaners,Soltero-a,Negro,Mujer,0,0,40,United-States,<=50K
570,24,Private,HS-grad,Never-married,Craft-repair,Hijo-unico,Blanco,Hombre,0,0,40,United-States,<=50K
673,33,Private,Bachelors,Married-civ-spouse,Exec-managerial,Marido,Blanco,Hombre,0,0,40,United-States,>50K


Nos encontramos con un 9.69% de filas duplicadas no es extremo, pero sí lo bastante alto como para que sea necesario revisar que hacer con ello:

- ¿Qué representan las filas?
El kaggle de la practica nos muestra que una fila esta compuesta por "12 columnas con información relevante para poder predecir el salario de una persona."

- Si cada fila representa una observación única los duplicados probablemente son errores y deberías eliminarlos.

- Si las filas representan eventos que pueden repetirse legítimamente, los duplicados podrían ser válidos, y eliminarlos podría hacerte perder información.

Analizando el significado del dataset es posible que distintas personas reales compartan exactamente las mismas características en las columnas del dataset (edad, trabajo, estado civil, etc.), especialmente si las variables son categóricas y discretas. Pero que tambien se repitan con

In [ ]:

df_train_sin_duplicados = df_train.drop_duplicates()
print(f"Filas en el dataset original: {len(df_train)}")
print(f"Filas después de eliminar duplicados: {len(df_train_sin_duplicados)}")

In [ ]:

df_train = df_train.drop_duplicates()
print("Dataset actualizado sin filas duplicadas")

# Limpieza de datos - Missing values
Buscamos y analizamos el porcentaje de NANs( valores vacios) que se encuentran en dataset, luego trataremos estos valores aplicando una estrategia:


Lo primero que nos fijamos es que la forma del dataset para representar los valores vacios, es a través del cáracter "?". Por lo que procederemos a verdaderamente vaciar dichos valores.

In [7]:
df_train.replace(r'^\s*\?\s*$', np.nan, regex=True, inplace=True)

In [8]:
print(df_train.isna().sum())

edad                        0
trabajo                  1568
estudios                    0
estado-civil                0
trabajo.1                1573
posicion-familiar           0
etnia                       0
sexo                        0
ganancias_inversiones       0
perdidas_inversiones        0
horas-trabajo_semana        0
pais-origen               495
salario                     0
dtype: int64


In [9]:
nan_por_columna = df_train.isnull().mean() * 100
pd.set_option('display.max_rows', None)
print(nan_por_columna[nan_por_columna > 0])

trabajo        5.600400
trabajo.1      5.618258
pais-origen    1.767983
dtype: float64


Solo estas 3 columnas poseen valores nulos.

In [19]:
nan_por_fila = df_train.isnull().mean(axis=1) * 100
print(nan_por_fila[nan_por_fila > 0].head(20))

14       7.692308
38       7.692308
51       7.692308
93       7.692308
245      7.692308
249      7.692308
393      7.692308
453      7.692308
557      7.692308
712      7.692308
725      7.692308
729      7.692308
777      7.692308
780      7.692308
887      7.692308
955      7.692308
1026     7.692308
1036     7.692308
1115     7.692308
1158     7.692308
1199     7.692308
1224     7.692308
1252     7.692308
1326     7.692308
1348     7.692308
1391     7.692308
1554     7.692308
1557     7.692308
1581     7.692308
1593     7.692308
1711     7.692308
1738     7.692308
1818     7.692308
1900     7.692308
1990     7.692308
2015     7.692308
2099     7.692308
2104     7.692308
2181     7.692308
2371     7.692308
2512     7.692308
2518     7.692308
2549     7.692308
2572     7.692308
2587     7.692308
2591     7.692308
2639     7.692308
2717     7.692308
2735     7.692308
2775     7.692308
2794     7.692308
2909     7.692308
2926     7.692308
3023     7.692308
3107     7.692308
3164     7

In [11]:
porcentaje_filas_nan = (df_train.isna().any(axis=1).mean()) * 100
print(f"Porcentaje de filas con algún un NaN: {porcentaje_filas_nan:.2f}%")

Porcentaje de filas con al menos un NaN: 7.30%


Estrategia para tratar los valores faltantes

Para tratar los valores faltantes, seguiremos la siguiente estrategia:

1. Para variables numéricas:
   - No es necesario tratar nada ya que los NANs solo aparecen en tres columnas categóricas
   
2. Para variables categóricas:
   - Debido a la proporción de filas con al menos un NANs, creemos que ese 7,3% es lo sufientemente notorio como para evitar eliminar todas esas filas, y buscar el imputar los valores, pero a su vez no es una gran parte del dataset por lo que podríamos prescindir de algunas filas.
   - Con esto en mente, hemos optado por usar dos estrategias al mismo tiempo:
        - Para las filas con 2 o más nulos. --> Se eliminan.
        - Para las filas con solo un nulo, rellenaremos ese valor a través de la moda (valor más frecuente) de su columna categórica.
   

Esta estrategia nos ayudará a mantener la distribución de los datos sin introducir sesgos significativos, a su vez que se eliminan aquellas filas con mucha información perdida.

In [12]:
# Primero, mantenemos solo las filas que tienen al menos (n_columnas - 1) valores no nulos
# Es decir: elimina las que tienen 2 o más NaN
df_train = df_train.dropna(thresh=df_train.shape[1] - 1)

In [13]:
# Segundo, separamos las columnas numéricas y categóricas
numeric_features = df_train.select_dtypes(include=['int64']).columns.tolist()
categorical_features = df_train.select_dtypes(include=['object']).columns.tolist()

print("Características numéricas:", numeric_features)
print("\nCaracterísticas categóricas:", categorical_features)

Características numéricas: ['edad', 'ganancias_inversiones', 'perdidas_inversiones', 'horas-trabajo_semana']

Características categóricas: ['trabajo', 'estudios', 'estado-civil', 'trabajo.1', 'posicion-familiar', 'etnia', 'sexo', 'pais-origen', 'salario']


A la hora de rellenar los NANs en las columnas, a su vez podemos aprovechar para realizar la codificación one-hot que hace columna binaria por categoría, ya que la inmensa mayoria de los modelos "solo entienden números" y no acepta operar directamente con texto.

¿Porque codificación One-hot?

Debido a que si las transformasemos a números los dintintos valores de una columna categórica como por ejemplo "trabajo":
Private → 1
State-gov → 2
Self-emp → 3
Los modelo interpretaría que “State-gov” está entre “Private” y “Self-emp”, lo cual no tiene sentido no hay orden natural entre categorías, solo serviría para columnas donde esta relacion si exista, y posibles valores son por ejemplo “bajo”, “medio”, “alto”.

El One-hot al crear columna binaria por categoría, rompe esta posible relación que no aplica.

In [14]:
# pipeline de preprocesamiento solo para variables categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # Usamos OneHotEncoder para transformar los datos categóricos
])

# Aplicamos la transformación solo a las columnas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'  # Esto mantiene las columnas numéricas sin modificar
)

# Ajustamos y transformamos los datos
transformed = preprocessor.fit_transform(df_train)
print(transformed.shape)

(26430, 105)


In [15]:
encoder = preprocessor.named_transformers_['cat'].named_steps['encoder']
encoded_cols = encoder.get_feature_names_out(categorical_features)
all_columns = list(encoded_cols) + numeric_features

print(all_columns)  # Número total de columnas después de la transformación

['trabajo_ Federal-gov', 'trabajo_ Local-gov', 'trabajo_ Never-worked', 'trabajo_ Private', 'trabajo_ Self-emp-inc', 'trabajo_ Self-emp-not-inc', 'trabajo_ State-gov', 'trabajo_ Without-pay', 'estudios_ 10th', 'estudios_ 11th', 'estudios_ 12th', 'estudios_ 1st-4th', 'estudios_ 5th-6th', 'estudios_ 7th-8th', 'estudios_ 9th', 'estudios_ Assoc-acdm', 'estudios_ Assoc-voc', 'estudios_ Bachelors', 'estudios_ Doctorate', 'estudios_ HS-grad', 'estudios_ Masters', 'estudios_ Preschool', 'estudios_ Prof-school', 'estudios_ Some-college', 'estado-civil_ Divorced', 'estado-civil_ Married-AF-spouse', 'estado-civil_ Married-civ-spouse', 'estado-civil_ Married-spouse-absent', 'estado-civil_ Never-married', 'estado-civil_ Separated', 'estado-civil_ Widowed', 'trabajo.1_ Adm-clerical', 'trabajo.1_ Armed-Forces', 'trabajo.1_ Craft-repair', 'trabajo.1_ Exec-managerial', 'trabajo.1_ Farming-fishing', 'trabajo.1_ Handlers-cleaners', 'trabajo.1_ Machine-op-inspct', 'trabajo.1_ Other-service', 'trabajo.1_ P

In [16]:
df_transformed = pd.DataFrame(transformed, columns=all_columns)
df_transformed.head()

,trabajo_ Federal-gov,trabajo_ Local-gov,trabajo_ Never-worked,trabajo_ Private,trabajo_ Self-emp-inc,trabajo_ Self-emp-not-inc,trabajo_ State-gov,trabajo_ Without-pay,estudios_ 10th,estudios_ 11th,...,pais-origen_ Trinadad&Tobago,pais-origen_ United-States,pais-origen_ Vietnam,pais-origen_ Yugoslavia,salario_ <=50K,salario_ >50K,edad,ganancias_inversiones,perdidas_inversiones,horas-trabajo_semana
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,39.0,2174.0,0.0,40.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,50.0,0.0,0.0,13.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,38.0,0.0,0.0,40.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,53.0,0.0,0.0,40.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,28.0,0.0,0.0,40.0


Hay que eliminar la columna salario_<=50k ya que es el inverso de salrio_>50K